In [1]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200918'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)
  !pip install pyngrok

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)


# Let's do all of our imports here, too.
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG
from ipywidgets import Text, Textarea, Layout
import plotly.express as px

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, ConnectMeshcatVisualizer, 
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MultibodyPlant, MultibodyPositionToGeometryPose, Parser, 
    PiecewisePolynomial, PiecewiseQuaternionSlerp, Quaternion, RigidTransform, 
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource
)
from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback

# TODO(russt): Move this to drake (adding the element name support to the base class).
import pandas as pd

def dataframe(trajectory, times, names):
  assert trajectory.rows() == len(names)
  values = trajectory.vector_values(times)
  data = {'t': times }
  for i in range(len(names)):
    data[names[i]] = values[i,:]
  return pd.DataFrame(data)

In [107]:
def grasp_poses_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step = 0.0)
    parser = Parser(plant, scene_graph)
    grasp = parser.AddModelFromFile(FindResourceOrThrow(
        "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "grasp")
    # TODO(russt): Draw the pregrasp gripper, too, as transparent (drake #13970).
    #pregrasp = parser.AddModelFromFile(FindResourceOrThrow(
    #    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "pregrasp")
    brick = parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/061_foam_brick.sdf"), "brick")
    plant.Finalize()
    
    frames_to_draw = {"grasp": {"body"}, "brick":{"base_link"}}

    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,frames_to_draw=frames_to_draw,
        axis_length=0.3,
        axis_radius=0.01)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    # TODO(russt): Set a random pose of the object.

    # Get the current object, O, pose
    B_O = plant.GetBodyByName("base_link", brick)
    #X_WO = plant.EvalBodyPoseInWorld(plant_context, B_O)
    #position brick
    p0_WO = [-0.2, -0.65, 0.12] # object in world frame
    R0_WO = RotationMatrix.MakeYRotation(np.pi/2)
    X_WO = RigidTransform(R0_WO, p0_WO)
    plant.SetFreeBodyPose(plant_context, B_O, X_WO)
    
    #get gripper handle
    B_G = plant.GetBodyByName("body", grasp)
    #position gripper
    #R0_WG =RotationMatrix.MakeXRotation(-np.pi/2)
    
    #p_GGO=[0,0.02,0.0] #position of object wrt gripper in gripper frame
    #X_G=RigidTransform(R0_WG)
    #X_
    #X_OG = X_G.multiply(X_WO.inverse())#.multiply(X_G)
    #X_WG= X_WG.multiply(X_WO)
    #tt=RigidTransform(p0_WO).multiply(RigidTransform(p_GWO))
    #X_OG=RigidTransform()
    #X_WG = X_WO.multiply(X_OG)
    #plant.SetFreeBodyPose(plant_context, B_G, X_G)
    

    #B_Ggrasp = plant.GetBodyByName("body", grasp)
    #p_GgraspO = [0, 0.12, 0]
    #R_GgraspO = RotationMatrix.MakeXRotation(np.pi/2.0).multiply(
    #    RotationMatrix.MakeZRotation(np.pi/2.0))
    #X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
    #X_OGgrasp = X_GgraspO.inverse()
    #X_WGgrasp = X_WO.multiply(X_OGgrasp)

    #plant.SetFreeBodyPose(plant_context, B_Ggrasp, X_WGgrasp)
    # Open the fingers, too.
    plant.GetJointByName("left_finger_sliding_joint", grasp).set_translation(plant_context, -0.054)
    plant.GetJointByName("right_finger_sliding_joint", grasp).set_translation(plant_context, 0.054)

    meshcat.load()
    diagram.Publish(context)

grasp_poses_example()

[2020-10-07 23:25:00.961] [console] [warning] Currently MultibodyPlant does not handle joint limits for continuous models. However some joints do specify limits. Consider setting a non-zero time step in the MultibodyPlant constructor; this will put MultibodyPlant in discrete-time mode, which does support joint limits.
[2020-10-07 23:25:00.962] [console] [warning] Joints that specify limits are: `left_finger_sliding_joint`, `right_finger_sliding_joint`.
[2020-10-07 23:25:00.965] [console] [warning] Currently MultibodyPlant does not handle joint limits for continuous models. However some joints do specify limits. Consider setting a non-zero time step in the MultibodyPlant constructor; this will put MultibodyPlant in discrete-time mode, which does support joint limits.
[2020-10-07 23:25:00.966] [console] [warning] Joints that specify limits are: `left_finger_sliding_joint`, `right_finger_sliding_joint`.


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6005...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Connected to meshcat-server.


In [106]:
def grasp_poses_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step = 0.0)
    parser = Parser(plant, scene_graph)
    grasp = parser.AddModelFromFile(FindResourceOrThrow(
        "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "grasp")
    # TODO(russt): Draw the pregrasp gripper, too, as transparent (drake #13970).
    #pregrasp = parser.AddModelFromFile(FindResourceOrThrow(
    #    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "pregrasp")
    brick = parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/061_foam_brick.sdf"), "brick")
    plant.Finalize()
    
    frames_to_draw = {"grasp": {"body"}, "brick":{"base_link"}}

    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,frames_to_draw=frames_to_draw,
        axis_length=0.3,
        axis_radius=0.01)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    # TODO(russt): Set a random pose of the object.

    # Get the current object, O, pose
    B_O = plant.GetBodyByName("base_link", brick)
    #X_WO = plant.EvalBodyPoseInWorld(plant_context, B_O)
    #position brick
    p0_WO = [-0.2, -0.65, 0.12] # object in world frame
    R0_WO = RotationMatrix.MakeYRotation(np.pi/2)
    X_WO = RigidTransform(R0_WO, p0_WO)
    plant.SetFreeBodyPose(plant_context, B_O, X_WO)
    

    

    B_Ggrasp = plant.GetBodyByName("body", grasp)
    p_GgraspO = [0, 0.12, 0]
    
    Rn=RotationMatrix.MakeZRotation(np.pi/2)
    R_GgraspO = R0_WO.multiply((Rn))
    X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
    X_OGgrasp = X_GgraspO.inverse()
    X_WGgrasp = X_WO.multiply(X_OGgrasp)

    plant.SetFreeBodyPose(plant_context, B_Ggrasp, X_WGgrasp)
    # Open the fingers, too.
    plant.GetJointByName("left_finger_sliding_joint", grasp).set_translation(plant_context, -0.054)
    plant.GetJointByName("right_finger_sliding_joint", grasp).set_translation(plant_context, 0.054)

    meshcat.load()
    diagram.Publish(context)

grasp_poses_example()

[2020-10-07 23:20:06.774] [console] [warning] Currently MultibodyPlant does not handle joint limits for continuous models. However some joints do specify limits. Consider setting a non-zero time step in the MultibodyPlant constructor; this will put MultibodyPlant in discrete-time mode, which does support joint limits.
[2020-10-07 23:20:06.775] [console] [warning] Joints that specify limits are: `left_finger_sliding_joint`, `right_finger_sliding_joint`.
[2020-10-07 23:20:06.778] [console] [warning] Currently MultibodyPlant does not handle joint limits for continuous models. However some joints do specify limits. Consider setting a non-zero time step in the MultibodyPlant constructor; this will put MultibodyPlant in discrete-time mode, which does support joint limits.
[2020-10-07 23:20:06.779] [console] [warning] Joints that specify limits are: `left_finger_sliding_joint`, `right_finger_sliding_joint`.


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6005...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Connected to meshcat-server.
